Option chains
=======

In [1]:
from ib_insync import *
util.startLoop()

ib = IB()
ib.connect('127.0.0.1', 7497, clientId=12)

<IB connected to 127.0.0.1:7497 clientId=12>

Suppose we want to find the options on the SPX, with the following conditions:

* Use the next three monthly expiries;
* Use strike prices within +- 20 dollar of the current SPX value;
* Use strike prices that are a multitude of 5 dollar.

To get the current market value, first create a contract for the underlyer (the S&P 500 index):

In [2]:
spx = Index('SPX', 'CBOE')
ib.qualifyContracts(spx)

[Index(conId=416904, symbol='SPX', exchange='CBOE', currency='USD', localSymbol='SPX')]

Then get the ticker. Requesting a ticker can take up to 11 seconds.

In [3]:
[ticker] = ib.reqTickers(spx)
ticker

Ticker(contract=Index(conId=416904, symbol='SPX', exchange='CBOE', currency='USD', localSymbol='SPX'), time=datetime.datetime(2019, 4, 15, 11, 45, 43, 178160, tzinfo=datetime.timezone.utc), bid=-1.0, bidSize=0, ask=-1.0, askSize=0, close=2907.41, ticks=[], tickByTicks=[], domBids=[], domAsks=[], domTicks=[])

Take the current market value of the ticker:

In [4]:
spxValue = ticker.marketPrice()
spxValue

2907.41

The following request fetches a list of option chains:

In [5]:
chains = ib.reqSecDefOptParams(spx.symbol, '', spx.secType, spx.conId)

util.df(chains)

,exchange,underlyingConId,tradingClass,multiplier,expirations,strikes
0,SMART,416904,SPX,100,"[20190417, 20190516, 20190620, 20190718, 20190...","[100.0, 200.0, 300.0, 400.0, 500.0, 600.0, 650..."
1,CBOE,416904,SPXW,100,"[20190415, 20190417, 20190418, 20190422, 20190...","[800.0, 900.0, 1000.0, 1050.0, 1100.0, 1150.0,..."
2,SMART,416904,SPXW,100,"[20190415, 20190417, 20190418, 20190422, 20190...","[800.0, 900.0, 1000.0, 1050.0, 1100.0, 1150.0,..."
3,CBOE,416904,SPX,100,"[20190417, 20190516, 20190620, 20190718, 20190...","[100.0, 200.0, 300.0, 400.0, 500.0, 600.0, 650..."


These are four option chains that differ in ``exchange`` and ``tradingClass``. The latter is 'SPX' for the monthly and  'SPXW' for the weekly options. Note that the weekly expiries are disjoint from the monthly ones, so when interested in the weekly options the monthly options can be added as well.

In this case we're only interested in the montly options trading on SMART:

In [6]:
chain = next(c for c in chains if c.tradingClass == 'SPX' and c.exchange == 'SMART')
chain

OptionChain(exchange='SMART', underlyingConId='416904', tradingClass='SPX', multiplier='100', expirations=['20190417', '20190516', '20190620', '20190718', '20190815', '20190919', '20191219', '20200116', '20200319', '20200618', '20201217', '20211216'], strikes=[100.0, 200.0, 300.0, 400.0, 500.0, 600.0, 650.0, 700.0, 750.0, 800.0, 850.0, 900.0, 950.0, 1000.0, 1050.0, 1100.0, 1150.0, 1175.0, 1200.0, 1225.0, 1250.0, 1275.0, 1300.0, 1325.0, 1350.0, 1375.0, 1400.0, 1425.0, 1450.0, 1475.0, 1500.0, 1525.0, 1550.0, 1575.0, 1600.0, 1625.0, 1650.0, 1660.0, 1670.0, 1675.0, 1680.0, 1690.0, 1700.0, 1710.0, 1720.0, 1725.0, 1730.0, 1740.0, 1750.0, 1760.0, 1770.0, 1775.0, 1780.0, 1790.0, 1800.0, 1810.0, 1820.0, 1825.0, 1830.0, 1840.0, 1850.0, 1860.0, 1870.0, 1875.0, 1880.0, 1885.0, 1890.0, 1895.0, 1900.0, 1905.0, 1910.0, 1915.0, 1920.0, 1925.0, 1930.0, 1935.0, 1940.0, 1945.0, 1950.0, 1955.0, 1960.0, 1965.0, 1970.0, 1975.0, 1980.0, 1985.0, 1990.0, 1995.0, 2000.0, 2005.0, 2010.0, 2015.0, 2020.0, 2025.0, 

What we have here is the full matrix of expirations x strikes. From this we can build all the option contracts that meet our conditions:

In [7]:
strikes = [strike for strike in chain.strikes
        if strike % 5 == 0
        and spxValue - 20 < strike < spxValue + 20]
expirations = sorted(exp for exp in chain.expirations)[:3]
rights = ['P', 'C']

contracts = [Option('SPX', expiration, strike, right, 'SMART', tradingClass='SPX')
        for right in rights
        for expiration in expirations
        for strike in strikes]

contracts = ib.qualifyContracts(*contracts)
len(contracts)

Started to throttle requests
Stopped to throttle requests


48

In [8]:
contracts[0]

Option(conId=350155156, symbol='SPX', lastTradeDateOrContractMonth='20190417', strike=2890.0, right='P', multiplier='100', exchange='SMART', currency='USD', localSymbol='SPX   190418P02890000', tradingClass='SPX')

Now to get the market data for all options in one go:

In [9]:
tickers = ib.reqTickers(*contracts)

tickers[0]

Started to throttle requests
Stopped to throttle requests


Ticker(contract=Option(conId=350155156, symbol='SPX', lastTradeDateOrContractMonth='20190417', strike=2890.0, right='P', multiplier='100', exchange='SMART', currency='USD', localSymbol='SPX   190418P02890000', tradingClass='SPX'), time=datetime.datetime(2019, 4, 15, 11, 45, 56, 310487, tzinfo=datetime.timezone.utc), bid=3.7, bidSize=195, ask=4.5, askSize=150, close=4.85, ticks=[], tickByTicks=[], domBids=[], domAsks=[], domTicks=[], bidGreeks=OptionComputation(impliedVol=0.10893876961775564, delta=-0.2737933733958956, optPrice=3.700000047683716, pvDividend=0.6298709657650186, gamma=0.010915584003424055, vega=0.9322332917839024, theta=-1.445380579317793, undPrice=2908.088623046875), askGreeks=OptionComputation(impliedVol=0.11920981403482983, delta=-0.29098407055268816, optPrice=4.5, pvDividend=0.6298709657650186, gamma=0.010271670105367125, vega=0.9599491743792606, theta=-1.6321370943102578, undPrice=2908.088623046875), lastGreeks=OptionComputation(impliedVol=None, delta=None, optPrice=

The option greeks are available from the ``modelGreeks`` attribute, and if there is a bid, ask resp. last price available also from ``bidGreeks``, ``askGreeks`` and ``lastGreeks``. For streaming ticks the greek values will be kept up to date to the current market situation.

In [10]:
ib.disconnect()